### 00 OpenAI API KEY 환경변수 등록

In [1]:
import os, json

with open('conf.json', 'r') as f:
        json_data = json.load(f)
    
os.environ['OPENAI_API_KEY'] = json_data['openai_config']["API_KEY"]

### 01 설정

In [2]:
class CONFIG:
    etf_tkr="AIQ"

In [ ]:
from data.get_data import get_pdf_df

pdf_df = get_pdf_df(etf_tkr=CONFIG.etf_tkr)

tkr_list = pdf_df.child_stk_tkr.to_list()

### 02 영향도 상위 5개 기업 추출

#### 02-1 2분기 가격데이터 로드

In [21]:
from data.get_data import get_prices_df
from data.fetch_data import fetch_data_from_db

prices_df = get_prices_df(tickers=tkr_list, start_date="2023-03-01")

#### 02-2 유동주식수 로드

In [64]:
import pandas as pd

sql = """
    SELECT * FROM os_stk_info;
"""

stk_info_df = pd.DataFrame(fetch_data_from_db(query=sql)).dropna()

stk_info_df = stk_info_df[stk_info_df["stk_tkr"].isin(prices_df.columns.str.upper().to_list())].reset_index(drop=True)

stk_info_df.index = stk_info_df['stk_tkr'].str.lower()

stk_info_df = stk_info_df[['float_shares']]

#### 02-3 영향도 상위 5개 기업 산출

In [63]:
시작_영향도 = stk_info_df['float_shares'] * prices_df.iloc[0]

끝_영향도 = stk_info_df['float_shares'] * prices_df.iloc[-1]

영향도 = abs(끝_영향도 - 시작_영향도)

영향도_top5_종목들_리스트 = 영향도.sort_values(ascending=False).head().index.str.upper().to_list()

영향도_top5_종목들_리스트

['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN']

### 03 뉴스데이터 로드

In [69]:
import yfinance as yf
from tqdm.auto import tqdm

stk_infos = yf.Tickers(영향도_top5_종목들_리스트)

display(tkr_list[0])

text = {
    f"{CONFIG.etf_tkr} news": []
}

for tkr in tqdm(영향도_top5_종목들_리스트):
    
    tmp = {}
    tmp["stock"] = tkr
    tmp["news_titles"] = []
    
    news_infos = stk_infos.tickers[tkr].news
    
    for news in news_infos:
        tmp['news_titles'].append(news["title"])
        
    text[f"{CONFIG.etf_tkr} news"].append(tmp)
    
text = str(text)

print(text)

'AAPL'

  0%|          | 0/5 [00:00<?, ?it/s]

{'AIQ news': [{'stock': 'AAPL', 'news_titles': ['Dow Jones Futures Rise For Ailing Market Rally; Tesla Stock Gets Huge Price Target Hike', 'SK Hynix’s $24 Billion Rally Unraveling on US-China Tech War', 'iPhone 15 launch: Release date, price and new features', 'Goldman Sachs China Stocks: Top 10 Stock Picks', 'Apple Stock, Adobe, Oracle, Moderna, Lennar, Inflation Data, and More to Watch This Week', 'A testing week for Apple and ECB rate-setters', 'Where Will Apple Stock Be in 3 Years?', 'Steamy Weather and a Too-Hot Economy: Your US Sunday Briefing']}, {'stock': 'MSFT', 'news_titles': ['2 Artificial Intelligence (AI) Stocks to Buy in September 2023 That Could Soar Like Nvidia', '13 Best Stocks To Invest In According to AI', 'Why Companies From Microsoft to Shopify Are Investing Millions in Carbon Removal', 'Why We Like The Returns At Microsoft (NASDAQ:MSFT)', 'Beleaguered Google faces day in court as Biden cracks down on monopolies', 'Artificial intelligence technology behind ChatGPT 

### 03 프롬프트 엔지니어링

In [70]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI


template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                f"""
                Summarize the common issues based on the news headlines of stocks which are best 5 influential components of {CONFIG.etf_tkr} in Korean.
                
                - Tone : Humble
                - Style : MECE, accurate
                - Reader level : college student
                - Length : within 500 characters
                - Perspective : Mckinsey&Company senior consultant
                - Format : markdown
                
                You MUST answer in Korean. 
                """
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

answer = llm(template.format_messages(text=text)).content

print(answer)

AIQ의 5개 핵심 구성 요소인 AAPL, MSFT, NVDA, GOOGL, AMZN 주식에 대한 뉴스 헤드라인을 분석한 결과, 공통적으로 다음과 같은 문제점이 나타났습니다.

1. 기술 전쟁과 관련된 불확실성: 미국과 중국의 기술 전쟁으로 인해 SK Hynix와 같은 기업의 주가가 하락하고 있습니다.
2. 인플레이션과 경기 둔화의 영향: Apple 주식과 같은 기업은 기후 변화와 경기 둔화로 인해 테스트 주간을 맞이하고 있습니다.
3. 인공지능 기업의 투자: Microsoft와 Shopify 등 기업들이 수백만 달러를 탄소 제거 등에 투자하고 있습니다.
4. 법적 문제와 관련된 위험: Google과 같은 기업은 바이든 대통령의 독점 조사로 법적 문제에 직면하고 있습니다.
5. 인공지능 기술의 성장과 관련된 기대: 인공지능 기술의 성장에 따라 Nvidia와 같은 기업의 주가 상승이 예상되고 있습니다.

이러한 문제점을 해결하고 기회를 포착하기 위해서는 기업들은 기술 전쟁에 대비하고, 경기 변동성에 대한 대비책을 마련해야 합니다. 또한 법적 조치와 관련된 위험을 경계하고, 인공지능 기술에 대한 투자를 지속적으로 추구해야 합니다.
